In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
from lxml import etree, html as lhtml

In [4]:
from tqdm import tqdm

In [5]:
from operator import attrgetter, itemgetter
import time
import sys

In [6]:
import re
import json

In [7]:
def process_price(dct):
    dct1 = dct.copy()
    if 'name' in dct1.keys():
        del dct1['name']
    dct1['ts'] = dct1['x']
    dct1['price'] = dct1['y']
    del dct1['x']
    del dct1['y']
    return dct1

In [8]:
def get_field(thing, name, dct, el_name):
    fields = thing.xpath(name)
    if len(fields):
        dct[el_name] = fields[0]
        
def get_page(path, n_attempts=5, t_sleep=1, **kwargs):
    for i in range(n_attempts):
        ret = requests.get(path, **kwargs)
        if ret.status_code==200:
            return ret
        time.sleep(t_sleep)
    return ret

In [10]:
def error_(s):
    with lock():
        print(s, file=sys.stderr)

In [11]:
def process_price(dct):
    dct1 = dct.copy()
    if 'name' in dct1.keys():
        del dct1['name']
    dct1['ts'] = dct1['x']
    dct1['price'] = dct1['y']
    del dct1['x']
    del dct1['y']
    return dct1

def process_one(info):
    path_o, idx = info[0], info[1]
    path = 'https://gg.deals' + path_o
    ret = get_page(path, n_attempts=5, t_sleep=1)
    if ret.status_code!=200:
        error_('Error: page {} was not read. Code {}'.format(page, ret.status_code))
        return
    dct = {}
    dct['url'] = path
    tree = lhtml.fromstring(ret.text)
    soup = BeautifulSoup(ret.text, 'lxml')
    get_field(tree, '//a[@class="active"]/span/text()', dct, 'name')
    get_field(tree, '//div[@class="game-info-image"]/img/attribute::src', dct, 'image')
    lnk = tree.xpath('//a[contains(@class, "game-link-widget")]/attribute::href')
    if len(lnk):
        mu = get_page(lnk[0], n_attempts=5, t_sleep=1)
        if mu.status_code!=200:
            error_('Error: market_url was not read. Code {}. Game {}'.format(mu.status_code, dct['name']))
        dct['market_url'] = mu.url
    details = tree.xpath('//div[@id="game-info-side"]')
    if len(details):
        details = details[0]
        get_field(details, '//div[@class="game-info-details-section game-info-details-section-release"]/p/text()', dct, "release_date")
        get_field(details, '//div[@class="game-info-details-section game-info-details-section-developer"]/p/text()', dct, "developer")
        get_field(details, '//a[@class="score-circle score-metascore"]/span/span/text()', dct, "metacritic_score")
        get_field(details, '//a[@class="score-circle score-userscore"]/span/span/text()', dct, "user_score")
        gens = details.xpath('//div[@id="game-info-genres"]/div/a/text()')
        if len(gens):
            dct["genres"] = gens
        
        tags = details.xpath('//div[@id="game-info-tags"]/div/a/text()')
        if len(tags):
            dct["tags"] = tags
            
        features = details.xpath('//div[@id="game-info-features"]/div/a/text()')
        if len(features):
            dct["features"] = features

    scores = tree.xpath('//div[@class = "score-col full"]')
    if len(scores):
        scores = scores[0]
        get_field(scores, '//span[contains(@class, "reviews-label")]/text()', dct, "review_label")
        rc = scores.xpath('//span[contains(@class, "semi-transparent")]/text()')
        if len(rc):
            dct["review_count"] = int(re.sub('[^0-9]', '',rc[0]))
    tmp = tree.xpath('//span[contains(@class, "reviews-label")]/attribute::title')
    if len(tmp):
        tmp = tmp[0]
        perc = re.search('%', tmp)#.span()[0]
        if perc is not None:
            perc = perc.span()[0]
            dct["review_positive_pctg"] = int(tmp[:perc])
    
    tmp = soup.find('section', {'id':'game-dlcs'})
    if tmp is not None:
        tmp = tmp.find_all('a', {'class':"game-info-title"})
    if tmp is not None:
        dct["dlcs"] = list(map(attrgetter('text'), tmp))
    tmp = soup.find('section', {'id':'game-packs'})
    if tmp is not None:
        tmp = tmp.find_all('a', {'class':"game-info-title"})
    if tmp is not None:
        dct["packs"] = list(map(attrgetter('text'), tmp))
    
    hist = get_page('https://gg.deals/ru/games/chartHistoricalData/{}/?hideKeyshops=1'.format(idx),
                    n_attempts=5, t_sleep=1, headers={'X-Requested-With': 'XMLHttpRequest'})
    if hist.status_code!=200:
        error_('Error: Prise history of {} was not read. Code {}'.format(dct['name'], hist.status_code))
    else:
        pr = json.loads(hist.text)
        if 'chartData' in pr.keys():
            if 'deals' in pr['chartData'].keys():
                dct["price_history"] = list(map(process_price, pr['chartData']['deals']))
    
    syss = soup.find('div', {'class':'game-requirements-tabs'})
    if syss is not None:
        syss = syss.find_all('a', {'role':"button"})
    if syss is not None:
        dct["pc_systems"] = list(map(attrgetter('text'), syss))
    tmp = soup.find('div', {'class':'game-info-actions'}).find_all('span', {'class':'count'})
    if len(tmp):
        #print(tmp)
        digs = []
        for i in list(map(attrgetter('text'), tmp)):
            if not i in digs:
                digs.append(i)
        #print(digs[0])
        #print(digs[1])
        #print(digs[2])
        if len(digs)==3:
            dct["wishlist_count"], dct["alert_count"], dct["owners_count"] = digs[0], digs[1], digs[2]
    return dct

In [12]:
import time

In [13]:
import gzip
import json
import codecs

from multiprocessing.dummy import Pool, Queue

links = []
ids = []
page = 1
while len(links)<300:
    ret = get_page("https://gg.deals/games/?sort=metascore&type=1",
                   n_attempts=5, t_sleep=1, params={'page': page,})
    if ret.status_code!=200:
        print('Error: page {} was not read. Code {}'.format(page, ret.status_code), file=sys.stderr)
    tree = lhtml.fromstring(ret.text)
    links += tree.xpath('//a[contains(@class, "full-link")]/attribute::href')
    ids += tree.xpath('//div[contains(@class, "grid-list")]/div/attribute::data-container-game-id')
    page += 1

links = links[:300]
ids = ids[:300]
    
info = list(zip(links, ids))
    
queue = Queue(len(info))   # очередь ссылок на книги
for i in info[::-1]:
    queue.put(i)

def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            record = process_one(queue.get())
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 300/300 [02:58<00:00,  1.68it/s]
